In [3]:
import transformers
import torch
import numpy as np

In [28]:
def sample_adpative_margin(batch_size:int,sort:bool=True):
    samples = torch.normal(0.3, 0.01**0.05, size=(batch_size,))
    samples = torch.min(torch.max(samples, torch.tensor(0.0001)), torch.tensor(0.5))
    if sort:
        sorted_samples = torch.sort(samples)[0]
    return sorted_samples


In [31]:
a=sample_adpative_margin(10,True)

In [32]:
a

tensor([1.0000e-04, 1.0000e-04, 1.0000e-04, 3.7814e-01, 5.0000e-01, 5.0000e-01,
        5.0000e-01, 5.0000e-01, 5.0000e-01, 5.0000e-01])

In [11]:
import torch

# Define the mean and variance
mean = 0.3
variance = 0.01

# Sample 10 values from the normal distribution
samples = torch.normal(mean, variance**0.5, size=(10,))

# Sort the samples in ascending order
sorted_samples = torch.sort(samples).values

sorted_samples

tensor([0.1033, 0.2439, 0.2554, 0.2646, 0.3172, 0.3488, 0.4003, 0.4103, 0.4357,
        0.4391])

In [38]:
from transformers import BertTokenizer
from torch.utils.data import Dataset,DataLoader
# 假设`tokenizer`是一个已经加载的BERT分词器
tokenizer = BertTokenizer.from_pretrained('/sharedata/models/Bert-Base-Chinese')

# 假设`batch_sentences`是一个包含多个句子的列表
batch_sentences = [[["你好，世界！", "BERT模型是强大的。"],
                   ["你好，世界！", "我最想打你了。"]],
                   
                   [["我很好","你是谁"],
                    ["我很好","你撒大苏打"]],
                     [["我很好","你是谁"],
                    ["我很好","你撒大苏打"]],
                     [["我很好","你是谁"],
                    ["我很好","你撒大苏打"]]]



In [49]:
len(batch_sentences)

4

In [96]:
class CD(Dataset):
    def __init__(self) -> None:
        self.tokenizer =tokenizer
        self.data = batch_sentences
        
        
    def __getitem__(self, index):
        return self.tokenizer(self.data[index],padding="max_length", truncation=True, return_tensors='pt',max_length=16)
    
    def __len__(self):
        return len(self.data)
cd = CD()

In [97]:
dl =DataLoader(cd,batch_size=2)

In [100]:
for data in dl:
    print(data.input_ids)
    print("================================")
    print(data.input_ids.reshape())
    break

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


tensor([[[ 101,  872, 1962, 8024,  686, 4518, 8013,  102,  100, 3563, 1798,
          3221, 2487, 1920, 4638,  102],
         [ 101,  872, 1962, 8024,  686, 4518, 8013,  102, 2769, 3297, 2682,
          2802,  872,  749,  511,  102]],

        [[ 101, 2769, 2523, 1962,  102,  872, 3221, 6443,  102,    0,    0,
             0,    0,    0,    0,    0],
         [ 101, 2769, 2523, 1962,  102,  872, 3054, 1920, 5722, 2802,  102,
             0,    0,    0,    0,    0]]])


In [62]:
batch_sentences[0]

[['你好，世界！', 'BERT模型是强大的。'], ['你好，世界！', '我最想打你了。']]

In [86]:

# Tokenize, add special tokens, padding, attention masks and segment ids
encoded_input = tokenizer(batch_sentences[0], padding="max_length", truncation=True, return_tensors='pt',max_length=64,add_special_tokens=True)

# encoded_input包含以下键：
# input_ids: token的ID
# attention_mask: 指示哪些是填充token
# token_type_ids: 句子片段的ID（对于单句输入，通常是全0）


In [87]:
encoded_input.input_ids

tensor([[ 101,  872, 1962, 8024,  686, 4518, 8013,  102,  100, 3563, 1798, 3221,
         2487, 1920, 4638,  511,  102,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0],
        [ 101,  872, 1962, 8024,  686, 4518, 8013,  102, 2769, 3297, 2682, 2802,
          872,  749,  511,  102,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0]])

In [88]:
result = tokenizer(batch_sentences[0], padding="max_length", truncation=True,max_length=64,add_special_tokens=True)


In [95]:
result.input_ids

AttributeError: 'list' object has no attribute 'shape'

In [92]:
tokenizer.decode(result.input_ids[0])

'[CLS] 你 好 ， 世 界 ！ [SEP] [UNK] 模 型 是 强 大 的 。 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [93]:
batch_sentences[0]

[['你好，世界！', 'BERT模型是强大的。'], ['你好，世界！', '我最想打你了。']]